In [7]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from skimage import exposure
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define constants
image_size = (200, 200)
orientations = 8
pixels_per_cell = (12, 12)
cells_per_block = (1, 1)

# Function to extract HOG features from an image
def extract_hog_features(image):
    fd, _ = hog(
        image,
        orientations=orientations,
        pixels_per_cell=pixels_per_cell,
        cells_per_block=cells_per_block,
        visualize=True,
    )
    return fd

# Function to load and preprocess images
def load_and_preprocess_images(folder_path):
    X = []
    y = []

    for class_label, class_name in enumerate(os.listdir(folder_path)):
        class_folder = os.path.join(folder_path, class_name)
        if os.path.isdir(class_folder):
            for image_filename in os.listdir(class_folder):
                if image_filename.endswith(".jpg") or image_filename.endswith(".png"):
                    image_path = os.path.join(class_folder, image_filename)

                    # Load and resize the image
                    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    image = cv2.resize(image, image_size)

                    # Extract HOG features
                    hog_features = extract_hog_features(image)

                    X.append(hog_features)
                    y.append(class_label)

            print(f"*** {class_name} - {len(os.listdir(class_folder))} images processed ***")

    return X, y

# Load and preprocess training and testing data
X_train, y_train = load_and_preprocess_images("dat/a2/data/train")
print("**TRAIN TERMINADO**")
X_test, y_test = load_and_preprocess_images("dat/a2/data/test")
print("**TEST TERMINADO**")

*** bedroom - 100 images processed ***
*** Coast - 100 images processed ***
*** Forest - 100 images processed ***
*** Highway - 100 images processed ***
*** industrial - 100 images processed ***
*** Insidecity - 100 images processed ***
*** kitchen - 100 images processed ***
*** livingroom - 100 images processed ***
*** Mountain - 100 images processed ***
*** Office - 100 images processed ***
*** OpenCountry - 100 images processed ***
*** store - 100 images processed ***
*** Street - 100 images processed ***
*** Suburb - 100 images processed ***
*** TallBuilding - 100 images processed ***
**TRAIN TERMINADO**
*** bedroom - 116 images processed ***
*** coast - 260 images processed ***
*** forest - 228 images processed ***
*** highway - 160 images processed ***
*** industrial - 211 images processed ***
*** insidecity - 208 images processed ***
*** kitchen - 110 images processed ***
*** livingRoom - 189 images processed ***
*** livingRoom (Case Conflict) - 154 images processed ***
*** moun

In [8]:
# Define the parameter grid for grid search
param_grid = {
    'C': [0.1, 1, 10, 100],  # Different values of the regularization parameter C
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Different kernel functions to try
    'gamma': [0.001, 0.01, 0.1, 1, 10]
}

# Create an SVM classifier
svm = SVC()

# Perform grid search with cross-validation
grid_search = GridSearchCV(svm, param_grid, cv=5, n_jobs=-1)  # cv is the number of cross-validation folds
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the SVM model with the best hyperparameters on the entire training set
best_svm = SVC(**best_params)
best_svm.fit(X_train, y_train)

# Make predictions on the test data
y_test_pred = best_svm.predict(X_test)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

KeyboardInterrupt: 